In [1]:
# Import dependencies
import os
from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-04-23 07:34:15.128563: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 07:34:15.135753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745386455.144037   13865 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745386455.146526   13865 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745386455.153098   13865 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 19 intra-op, 12 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1745386456.361291   13865 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "DenseNetV1"
model_name      = f"{set_name}_2"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

class_weights = calculate_class_weights(train_ds, num_classes)

I0000 00:00:1745386456.381483   13865 cuda_executor.cc:479] failed to allocate 18.00PiB (20266198323167232 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745386456.381506   13865 cuda_executor.cc:479] failed to allocate 16.20PiB (18239577202360320 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745386456.381508   13865 cuda_executor.cc:479] failed to allocate 14.58PiB (16415619482124288 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745386456.381511   13865 cuda_executor.cc:479] failed to allocate 13.12PiB (14774057211789312 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745386456.381513   13865 cuda_executor.cc:479] failed to allocate 11.81PiB (13296650738991104 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745386456.381514   13865 cuda_executor.cc:

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

2025-04-23 07:35:25.478671: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
2025-04-23 07:35:28.939525: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 71842 samples [00:36, 1979.58 samples/s]
Counting samples: 15431 samples [00:07, 1977.19 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryCrossentropy(from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

In [5]:
# Model Training
model          = build_densenet121(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-23 07:35:38 - INFO - 
=== Training Started ===

2025-04-23 07:35:38 - INFO - Batch Size: 128
2025-04-23 07:35:38 - INFO - Optimizer: Adam
2025-04-23 07:35:38 - INFO - 

2025-04-23 07:35:38 - INFO - 
=== Starting Epoch 1 ===



Epoch 1/100


I0000 00:00:1745386546.711023   13932 service.cc:152] XLA service 0x79f534002760 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745386546.711039   13932 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1745386546.713775   13932 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745386546.755141   13932 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-23 07:36:06.380659: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 21119 of 25000
2025-04-23 07:36:08.279170: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-04-23 07:36:10.582820: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to 

311/561 ━━━━━━━━━━━━━━━━━━━━ 59s 238ms/step - accuracy: 0.8906 - auc: 0.5237 - f1_score: 0.0504 - loss: 2.2382 - precision: 0.1166 - recall: 0.0401 

2025-04-23 07:38:23.782274: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


...

561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.9142 - auc: 0.8016 - f1_score: 0.1479 - loss: 0.4187 - precision: 0.5053 - recall: 0.1061 

2025-04-23 12:13:14 - INFO - 
=== Epoch 100 Summary ===
2025-04-23 12:13:14 - INFO - Time: 163.18s
2025-04-23 12:13:14 - INFO - Training   - accuracy: 0.9145 - auc: 0.8025 - f1_score: 0.1477 - loss: 0.4176 - precision: 0.5049 - recall: 0.1070 - learning_rate: 0.0000
2025-04-23 12:13:14 - INFO - Validation - accuracy: 0.9116 - auc: 0.7407 - f1_score: 0.1185 - loss: 0.5125 - precision: 0.4080 - recall: 0.0834
2025-04-23 12:13:14 - INFO - ==================================================



561/561 ━━━━━━━━━━━━━━━━━━━━ 163s 291ms/step - accuracy: 0.9142 - auc: 0.8016 - f1_score: 0.1479 - loss: 0.4187 - precision: 0.5053 - recall: 0.1061 - val_accuracy: 0.9116 - val_auc: 0.7407 - val_f1_score: 0.1185 - val_loss: 0.5125 - val_precision: 0.4080 - val_recall: 0.0834 - learning_rate: 1.0000e-06


2025-04-23 12:13:14 - INFO - 
=== Training Completed! ===

2025-04-23 12:13:14 - INFO - Final Metrics: accuracy: 0.9145 - auc: 0.8025 - f1_score: 0.1477 - loss: 0.4176 - precision: 0.5049 - recall: 0.1070 - val_accuracy: 0.9116 - val_auc: 0.7407 - val_f1_score: 0.1185 - val_loss: 0.5125 - val_precision: 0.4080 - val_recall: 0.0834



Restoring model weights from the end of the best epoch: 98.
